# Regularization

We use a custom generated dataset for regularization to shwo the effect to regularization.

## Dataset

In [ ]:
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Use the helper code below to generate the data

# Defines the number of data points to generate
num_points = 100

# Generate predictor points (x) between 0 and 5
x = np.linspace(0, 5, num_points)

# Generate the response variable (y) using the predictor points
y = x * np.sin(x) + np.cos(x) + np.random.normal(loc=0, scale=1, size=num_points)

# Generate data of the true function y = x*sin(x) 
# x_b will be used for all predictions below 
x_b = np.linspace(0, 5, 100)
y_b = x_b * np.sin(x_b) + np.cos(x_b)

In [ ]:
# Split the data into train and test sets with .33 and random_state = 42
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
x_train = np.reshape(x_train, newshape=(-1, 1)).astype(np.float32)
x_test = np.reshape(x_test, newshape=(-1, 1)).astype(np.float32)
y_train = np.reshape(y_train, newshape=(-1, 1)).astype(np.float32)
y_test = np.reshape(y_test, newshape=(-1, 1)).astype(np.float32)

In [ ]:
plt.rcParams["figure.figsize"] = (10,8)

# Plot the train data
plt.plot(x_train,y_train, '.', label='Train data', markersize=15, color='#FF9A98')

# Plot the test data
plt.plot(x_test,y_test, '.', label='Test data', markersize=15, color='#75B594')

# Plot the true data
plt.plot(x_b, y_b, '-', label='True function', linewidth=3, color='#5E5E5E')

# Set the axes labels
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

## Create a neural network (sequential model)

In [ ]:
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(1, 128),
    nn.Linear(128, 128),
    nn.Linear(128, 128),
    nn.Linear(128, 1)
)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

c:\Users\roessle\Anaconda3\envs\aai_2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Please use the training and evaluation methods for your model(s)

import torch
torch.manual_seed(42)
from torch.utils.data import DataLoader, TensorDataset

def train_model(
    model, optimizer, loss_fn, x_dataset, y_dataset, epochs, batch_size
):
    data_loader = DataLoader(dataset=TensorDataset(torch.as_tensor(x_dataset), torch.as_tensor(y_dataset)), batch_size=batch_size, shuffle=True)
    for epoch in range(0, epochs):
        for data in data_loader:
            x, y = data
            output = model(x)

            optimizer.zero_grad()
            loss = loss_fn(output, y)
            loss.backward()

            optimizer.step()
        
        if epoch % 100 == 0:
            print(f'Epoch: {epoch}; Loss: {loss}')
    
    return model


def eval_model(
    model, x_train, y_train, x_test
):
    x_train = torch.as_tensor(x_train)
    y_train = torch.as_tensor(y_train)

    x_test = torch.as_tensor(x_test)
    # y_test = torch.as_tensor(y_test)
    
    train_output = model(x_train).detach()
    test_output = model(x_test).detach()

    plt.rcParams["figure.figsize"] = (10,8)

    # Plot the true train data
    plt.plot(x_train, y_train, '.', label='Train true data', markersize=15, color='#000000')

    # Plot the predicted train data
    plt.plot(x_train, train_output, '.', label='Train predictions', markersize=15, color='#FF9A98')

    # Plot the test predictions
    plt.plot(x_test, test_output, '.', label='Test predictions', markersize=15, color='#75B594')

    # Plot the true data
    plt.plot(x_b, y_b, '-', label='True function', linewidth=3, color='#5E5E5E')

    # Set the axes labels
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.show()
        

## Initialize an unregularized network

In [ ]:
# Init your model

import torch.optim as optim
import torch.nn.functional as F

# create your model
unregularized_model = nn.Sequential(
    nn.Linear(1, 128),
    nn.Linear(128, 128),
    nn.Linear(128, 128),
    nn.Linear(128, 1)
)
print(unregularized_model)

# Init the adam optimizer
optimizer = optim.Adam(unregularized_model.parameters(), lr=0.01)
# Define MSE loss as loss function
loss_fn = nn.MSELoss()

### Train and evalute the model

In [ ]:
# use a batch size of 10
# use 1500 epochs for training

# use the train_model method to train the model
unregularized_model = train_model(
    model=unregularized_model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    x_dataset=x_train,
    y_dataset=y_train,
    epochs=1500,
    batch_size=10
)

# use the eval_model method to plot the results
eval_model(unregularized_model, x_train, y_train, x_test)

## Initialize a neural network with dropout

In [ ]:
# Init your model


import torch.optim as optim
import torch.nn.functional as F

# create your model
dropout_model = nn.Sequential(
    nn.Linear(1, 128),
    nn.Dropout(0.5),
    nn.Linear(128, 128),
    nn.Dropout(0.5),
    nn.Linear(128, 128),
    nn.Dropout(0.5),
    nn.Linear(128, 1)
)
print(dropout_model)

# Init the adam optimizer
optimizer_dropout = optim.Adam(dropout_model.parameters(), lr=0.01)
# Define MSE loss as loss function
loss_fn_dropout = nn.MSELoss()

### Train and evalute the model

In [ ]:
# use a batch size of 10
# use 1500 epochs

# use the train_model method to train the model
dropout_model = train_model(
    model=dropout_model,
    optimizer=optimizer_dropout,
    loss_fn=loss_fn_dropout,
    x_dataset=x_train,
    y_dataset=y_train,
    epochs=1500,
    batch_size=10
)

dropout_model.eval()
# use the eval_model method to plot the results
eval_model(dropout_model, x_train, y_train, x_test)

## Initialize a model with L2 regularization

In [ ]:
# Init your model

import torch.optim as optim
import torch.nn.functional as F

# create your model
l2_model = nn.Sequential(
    nn.Linear(1, 128),
    nn.Linear(128, 128),
    nn.Linear(128, 128),
    nn.Linear(128, 1)
)
print(l2_model)

# Init the adam optimizer
    # L2 regularization can be implemented by a parameter of the optimizer (no need to do it manually)
optimizer_l2 = optim.Adam(l2_model.parameters(), lr=0.01, weight_decay=0.01)
# Define MSE loss as loss function
loss_fn_l2 = nn.MSELoss()

### Train and evaluate the model

In [ ]:
# use a batch size of 10
# use 1500 epochs

# use the train_model method to train the model
l2_model = train_model(
    model=l2_model,
    optimizer=optimizer_l2,
    loss_fn=loss_fn_l2,
    x_dataset=x_train,
    y_dataset=y_train,
    epochs=1500,
    batch_size=10
)

# use the eval_model method to plot the results
eval_model(l2_model, x_train, y_train, x_test)

## Initialize a model with L1 regularization

In [ ]:
# Init your model

import torch.optim as optim
import torch.nn.functional as F

# create your model
l1_model = nn.Sequential(
    nn.Linear(1, 128),
    nn.Linear(128, 128),
    nn.Linear(128, 128),
    nn.Linear(128, 1)
)
print(l1_model)


# Init the adam optimizer
    # L1 regularization has to be implemented manually. There is no pre-defined function to use.
optimizer_l1 = optim.Adam(l1_model.parameters(), lr=0.01)
# Define MSE loss as loss function and add the l1 regularization
    # You have to access all the weights from the network (see previous exercises on how to do this)
l1_lambda = 0.001

def loss_fn_l1(output, target):
    mse_loss = nn.MSELoss()(output, target)
    l1_reg = torch.tensor(0., requires_grad=True)
    for param in l1_model.parameters():
        l1_reg = l1_reg + torch.norm(param, 1)
    return mse_loss + l1_lambda * l1_reg

### Train and evaluate the model

In [ ]:
# use a batch size of 10
# use 1500 epochs

# use the train_model method to train the model
l1_model = train_model(
    model=l1_model,
    optimizer=optimizer_l1,
    loss_fn=loss_fn_l1,
    x_dataset=x_train,
    y_dataset=y_train,
    epochs=1500,
    batch_size=10
)

# use the eval_model method to plot the results
eval_model(l1_model, x_train, y_train, x_test)